### Quantification of cell's characteristics. 

Here we create a data base of cell's characteristics. 

We use as an input the data base that was use for cell pose, check that both green and red images exists and also that the mask had been extracted.

We compute characteristics for the cells extracted in the masks.

In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from cellpose import utils, io
import czifile as czifile
import pandas as pd
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    # Warning-causing lines of code here
from PIL import Image

In [2]:
### Define all relevant directories and paths
directory_path = '/home/melma31/Documents/campari/'

### Input data base with mice data
data_base_path = directory_path + 'data_base.xlsx'

### Output data base with cell data. Backup copy is done simultaneusly (this is in case the code breaks while saving data base)
data_base_path_output = directory_path + 'data_base_output.xlsx'
data_base_path_output_bkup = directory_path + 'data_base_output_bkup.xlsx'

In [4]:
### If only some animals, days, side what to be analyzed, select them in here
MOUSE = None
SIDE = None
EXAMPLE = None
DATE = None


### Parameters for CELLPOSE AND FOR POSTERIOR SELECTION
CELLSIZE = 250

In [6]:
mouse_data = pd.read_excel(data_base_path)
cell_data_output = pd.read_excel(data_base_path_output)
print(mouse_data)
#print(cell_data_output)
#mouse_number = 1 
#mouse_data = dbase.loc[dbase['mouse_id'] == mouse_number]

### PART 2: SELECT DESIRED DATA TO ANALYSE
# Select only red channel data
#selected_data = mouse_data.loc[mouse_data['date'] == DATE]
#selected_data = selected_data.loc[selected_data['mouse_id'] == MOUSE]
#selected_data = selected_data.loc[selected_data['example'] == EXAMPLE]
#selected_data = selected_data.loc[selected_data['side'] == SIDE]

selected_data = mouse_data
mouse_data = selected_data

     date  month  year  mouse_id  condition  stimuli  side  example  \
0      11      1  2023         1          1        1     0        1   
1      11      1  2023         1          1        1     0        2   
2      11      1  2023         1          1        1     0        3   
3      11      1  2023         1          1        1     0        4   
4      11      1  2023         1          1        1     0        5   
..    ...    ...   ...       ...        ...      ...   ...      ...   
96     25      1  2023         2          1        2     0        6   
97     25      1  2023         2          1        2     1        1   
98     25      1  2023         2          1        2     1        2   
99     25      1  2023         2          1        2     1        3   
100    25      1  2023         2          1        2     1        4   

    green_file_name red_file_name  
0             GREEN           RED  
1             GREEN           RED  
2             GREEN           RED  
3  

In [7]:
def append_to_or_merge_with_states_df(states_df, inp):
    '''
    If the row(s) exist(s) in the analysis states dataframe already, replace it
    If it doesn't, append it to the analysis states dataframe.
    Args:
        inp: pd.Series object or pd.DataFrame object
        states_df: pd.DataFrame object
    Returns:
        states_df: pd.DataFrame object
            Analysis states dataframe with rows appended
    '''

    if str(type(inp)) == "<class 'pandas.core.frame.DataFrame'>":
        # If a dataframe is inserted, apply the function recursively
        for index, row in inp.iterrows():
            states_df = append_to_or_merge_with_states_df(states_df, row)
    else:
        # If a row is inserted
        if inp.name in states_df.index:
            # Replace the row in the analysis states dataframe
            logging.debug(f'Replacing row {inp.name} in analysis states dataframe')
            for item, value in inp.iteritems():
                states_df.loc[inp.name, item] = value
        else:
            logging.debug(f'Appending row {inp.name} to analysis states dataframe')
            # Append it to the analysis states dataframe
            states_df = states_df.append(inp)

    return states_df

In [8]:
### create output dataframe
cell_counter= len(cell_data_output)
for i in range(len(selected_data)):
    cell_data_output = pd.read_excel(data_base_path_output)
    cell_data_output.to_excel(data_base_path_output_bkup)  
    
    date = selected_data.iloc[i].date
    month =  selected_data.iloc[i].month
    year =  selected_data.iloc[i].year
    mouse = selected_data.iloc[i].mouse_id
    folder_structure = str(date) + '.' + str(month) + '.'+ str(year) + '/animal_' + str(mouse)
    side = selected_data.iloc[i].side
    if side == 0:
        folder_structure = folder_structure + '/left/'
    else:
        folder_structure = folder_structure + '/right/'
    example = selected_data.iloc[i].example
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].red_file_name
    filesname_red = file_path + '.tif'
    filesname_mask = file_path + '_cp_masks.png'
    file_path = directory_path +'data/' + folder_structure + str(example) + '_' + selected_data.iloc[i].green_file_name
    filesname_green = file_path + '.tif'
    
    if not os.path.exists(filesname_red) or not os.path.exists(filesname_green) or not os.path.exists(filesname_mask):
        print('Not all files exist')
    else:
        img_red = io.imread(filesname_red)
        masks = io.imread(filesname_mask)
        img_green = io.imread(filesname_green)
        
        n_cells = np.max(masks)
        mean_red = np.mean(img_red)
        mean_green = np.mean(img_green)
        #red_bkg.append(mean_red)
        #green_bkg.append(mean_green)
        for cell in range(1,n_cells):
            cell_mask = np.where(masks == cell)
            cellsize = len(cell_mask[0])
            green_intensity = np.mean(img_green[cell_mask])-mean_green
            red_intensity = np.mean(img_red[cell_mask])-mean_red
            
            if cellsize >= 250:
                info_dictionary = { 'date':date,
                                   'month':month,
                                   'year':year,
                                   'mouse_id':mouse,
                                   'condition':selected_data.iloc[i].condition,
                                   'stimuli':selected_data.iloc[i].stimuli,
                                   'side':side,
                                   'example':example,
                                   'green_file_name':filesname_green,
                                   'red_file_name':filesname_red,
                                   'cell_id':cell,
                                   'cell_size': cellsize,
                                   'green_int':green_intensity,
                                   'red_int':red_intensity}
                new_df = pd.Series(info_dictionary)
                new_df.name = cell_counter + cell
                cell_data_output = append_to_or_merge_with_states_df(cell_data_output,new_df)
            cell_data_output.to_excel(data_base_path_output, index=False)
cell_data_output = cell_data_output.drop_duplicates()
cell_data_output.to_excel(data_base_path_output, index=False)

2023


397347